In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from helpers import *

In [ ]:
path = Path('./scratch')
folder='train2'
data_dir = path/'cvs'
file_format = 'cv-test_seed-{}-train_seed-{}.xlsx'

y_preds_per_validation = []
y_target_per_validation = []


test_seed = test_seeds[0]
for train_val_seed in train_seeds:
    data_file = data_dir / file_format.format(test_seed, train_val_seed)
    df_train_val, _ = nsea_load_data_from_excel(data_file)
    data=create_db_train_val(df_train_val, path, folder)
    model_name = "resnet152-full-model-test_seed-{}-train_seed-{}".format(test_seed, train_val_seed)
    y_preds_valid, _ = nsea_get_preds(model_name, data, None)
    y_preds_per_validation.append(y_preds_valid[0])
    y_target_per_validation.append(y_preds_valid[1])
    


In [ ]:
model = nsea_load_learner(model_name,data)

# Combine predictions and target to estimate optimal threshold

In [ ]:
y_preds = torch.cat(y_preds_per_validation) # Concat predictions
y_target = torch.cat(y_target_per_validation) # Concat targets

In [ ]:
y_preds_valid = [y_preds, y_target]

In [ ]:
thresholds = nsea_compute_thresholds(y_preds_valid)

In [ ]:
thresholds

In [ ]:
import pickle
output = open('resnet152-optimal-thresholds.pkl', 'wb')
pickle.dump(thresholds, output)
output.close()

In [ ]:
y_preds.shape

In [ ]:
from sklearn.metrics import average_precision_score, precision_recall_curve
import numpy

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (20,8)
font = {'family' : 'DejaVu Sans', 'weight' : 'normal', 'size'  : 22}
plt.rc('font', **font)

In [ ]:
precisions = dict()
recalls = dict()
Thresholds = dict()

In [ ]:
y_true = y_target.numpy()
y_pred = y_preds.numpy()

In [ ]:
for i in range(5):
    precisions[i], recalls[i], Thresholds[i] = precision_recall_curve(y_true[:, i], y_pred[:, i])

In [ ]:
opt_id = []
for i in range(5): 
    re = recalls[i]
    pre = precisions[i]
    dist = [ np.sqrt((1-re)**2 + (1-pre)**2) for re, pre in zip(re, pre) ]
    opt_id.append(dist.index(min(dist)))
    t = Thresholds[i]
    opt_thres = t[opt_id[i]]
    f1_score_opt = 2.0*re[opt_id[i]]*pre[opt_id[i]] / (re[opt_id[i]]+pre[opt_id[i]])
    print(re[opt_id[i]], pre[opt_id[i]], f1_score_opt)
    print("Optimal ",classes[i]," Threshold = ", opt_thres )
    print()

In [ ]:
Thresholds

In [ ]:
recalls

In [ ]:
precisions

In [ ]:
cmaps = OrderedDict()
plt.figure(figsize=(12,8))
for i in range(5): 
    plt.plot(recalls[i][3:-4], precisions[i][3:-4], label = classes[i], linewidth=3.0)
    #plt.plot([recalls[i][opt_id[i]],1],[precisions[i][opt_id[i]],1], 'ro--')
    plt.scatter(recalls[i][opt_id[i]],precisions[i][opt_id[i]], marker='*', s=900)
#     plt.text(0.85, 0.8, 'opt. threshold', color='r', size=18)
plt.xlabel('Recall', size=20)
plt.ylabel('Precision', size=20)
plt.xlim(0.75,1)
plt.ylim(0.75,1)
plt.title("Precision Recall Curves")
#plt.legend()
#plt.axis('off')
#plt.axis('equal')
#plt.gca().set_aspect('equal')
plt.savefig('figures/precision_recall_curve_zoom.png', dpi=300)
plt.show()

In [ ]:
thresholds

In [ ]:
df_thresholds = pd.DataFrame({'Threshold': thresholds}).T
df_thresholds

# Table 1

In [ ]:
print(df_thresholds.to_latex(float_format=lambda x: '%.3f' % truncate_decimals(x,3)))

# Get performance on each fold using the thresholds computed above

In [ ]:
thresholds

In [ ]:
new_compute_metrics(y_target_per_validation[0], y_preds_per_validation[0], thresholds)

In [ ]:
res=[]
for fold_idx,fold in enumerate(zip(y_preds_per_validation, y_target_per_validation)):
    print('Compute Performance on fold #', fold_idx)
    result = new_compute_metrics(fold[1], fold[0], thresholds)
    #write_dictionary_to_csv('results/resnet34-validation-metrics-{}-{}.csv'.format(test_seed,train_seeds[fold_idx]), res)
    print(test_seed, train_seeds[fold_idx])
    display(result)
    res.append(result)
    print('='*60)
    #print(res)

In [ ]:
df=pd.concat(res)

In [ ]:
df

In [ ]:
table_summary=df.groupby('Event').agg({
    'Threshold': 'mean',
    'Exact Matching Score': ['mean', 'std'], 
    'Recall': ['mean', 'std'], 
    'Precision': ['mean', 'std'], 
    'F1-Score': ['mean', 'std']
})

# Table 3

In [ ]:
print(table_summary.to_latex(float_format=lambda x: '%.3f' % truncate_decimals(x,3)))

In [ ]:
df.reset_index(inplace=True, drop=True)

In [ ]:
df_folds=df.query('Event=="Aggregate"')
df_folds.reset_index(drop=True, inplace=True)
df_folds.index = df_folds.index+1
df_folds.index.name='Fold #'
df_folds

# Table 2

In [ ]:
print(df_folds[['Exact Matching Score', 'Precision', 'Recall', 'F1-Score']].to_latex(float_format=lambda x: '%.3f' % truncate_decimals(x,3)))